In [2]:
# 체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
# 토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
# 모델 입력값 만들기
# encode() 메서드는 입력 문장을 토큰화한 뒤 정수로 인덱싱하는 역할 수행
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [5]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

In [10]:
# 그리디 서치로 문장 생성
import torch
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,  # 확률값이 높은 단어를 다음 단어로 결정
        min_length=10,
        max_length=50,
    )

In [11]:
# 토큰 인덱스를 문장으로 복원하기
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [12]:
# 빔 서치
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        num_beams=3,  # 빔 크기를 지정
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


In [13]:
# 반복 줄이기 1
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


In [14]:
# 반복 줄이기 2
# repetition penalty가 1.0이면, greedy search와 다르지 않음
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [15]:
# repetition_penalty=1.1
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [17]:
# repetition_penalty=1.2
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [18]:
# repetition_penalty=1.5
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


In [25]:
# 탑-k 샘플링: 확률 상 상위에 위치하는 k개의 토큰 중 하나를 랜덤으로 취함
with torch.no_grad():
    generate_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


In [26]:
# 템퍼러쳐 스케일링: 확률 분포를 대소 관계의 역전 없이 분포의 모양만을 바꿔서 문장을 다양하게 생성하는 기법
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [27]:
# 템퍼러쳐 값이 1보다 커질수록 다양한 문장이 생성되지만 품질은 나빠질 수 있다
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요"라는 제목으로 쓴 사진입니다.
그는 북한 주민들의 눈까지 수그러져 있다고 강조하면서도 북한 당국은 탈선을 의심하기는 할 수밖에요.
미 연방우주청은 이미 그 가능성을 거론하기는 한 발 이른 모습들을 취합니다.
대개의 일반 시민이 북한에서


In [28]:
# 탑-p 샘플링(nucleus sampling)
# 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이 p 이상인 최소 개수의 토큰 집합 가운데 하나를 선택
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!!!!!!!
#강아지스타그램 #멍스타그램 #펫스타그램 #petstagram #dogstagram #doggy #포메라니


In [29]:
# top-p=0.01
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요...
제가 좀 미안해요..
저도 너무 #기분좋은한하루였어요 ㅠㅠㅠㅠ 
그리고 이번에 너무 좋은 하루 보냈으니 내년에도 기분좋은하루 보내세요!!



In [30]:
# 종합 적용
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        temperature=0.9,
        top_k=50,
        top_p=0.92,
    )

In [31]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"라고 묻는다면 이 또한 당신은 '내 생각'일 것이다.
그리고 그 답변들은 대부분 거짓말이다.
그것은 마치 '이렇게 해야 한다'는 것이 아니라, '그렇다'의 뜻임을 깨닫지 못한 채 침묵하기만
